# Create Winter Year Snow Cubes from IMS
- Saving seperately from annual Canada IMS cubes because of memory issues down the line on cleaning...

In [9]:
import xarray as xr

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59911,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:59966,Total threads: 10
Dashboard: http://127.0.0.1:59980/status,Memory: 0 B
Nanny: tcp://127.0.0.1:59914,


2024-10-29 16:27:42,163 - distributed.nanny - ERROR - 
Traceback (most recent call last):
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\utils.py", line 832, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\nanny.py", line 552, in _on_worker_exit
    await self._unregister()
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\nanny.py", line 305, in _unregister
    await wait_for(
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\utils.py", line 1956, in wait_for
    return await fut
           ^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\core.py", line 1395, in send_recv_from_rpc
    return await send_recv(comm=comm, op=key, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\sno

# Load IMS cubes

In [37]:
year1 = 2022
year2 = 2023

path = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\2 IMS\ims_nc\IMS_'

In [38]:
%%time
c1 = pputil.uploadNC(path + str(year1) + '.nc', type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
c1

CPU times: total: 0 ns
Wall time: 41.2 ms


<xarray.DataArray (time: 365, y: 73, x: 47)> Size: 1MB
dask.array<open_dataset-__xarray_dataarray_variable__, shape=(365, 73, 47), dtype=uint8, chunksize=(1, 73, 47), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 376B 4.002e+05 4.012e+05 ... 4.452e+05 4.462e+05
  * y            (y) float64 584B 7.679e+06 7.678e+06 ... 7.608e+06 7.607e+06
  * time         (time) datetime64[ns] 3kB 2022-01-01 2022-01-02 ... 2022-12-31
    variable     <U18 72B ...
    spatial_ref  int64 8B ...
Attributes: (12/50)
    _FillValue:                 0
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ...                         ...
    metadata_link:              https://nsidc.org/data/G02156
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2022 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...

In [39]:
%%time
c2 = pputil.uploadNC(path + str(year2) + '.nc', type = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
c2

CPU times: total: 0 ns
Wall time: 107 ms


<xarray.DataArray (time: 365, y: 73, x: 47)> Size: 1MB
dask.array<open_dataset-__xarray_dataarray_variable__, shape=(365, 73, 47), dtype=uint8, chunksize=(1, 73, 47), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 376B 4.002e+05 4.012e+05 ... 4.452e+05 4.462e+05
  * y            (y) float64 584B 7.679e+06 7.678e+06 ... 7.608e+06 7.607e+06
  * time         (time) datetime64[ns] 3kB 2023-01-01 2023-01-02 ... 2023-12-31
    variable     <U18 72B ...
    spatial_ref  int64 8B ...
Attributes: (12/50)
    _FillValue:                 0
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ...                         ...
    metadata_link:              https://nsidc.org/data/G02156
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2023 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...

# Convert to snow cube
- Using Mean Snow Minimum for area of July 24 (205)

In [40]:
# %%time
# snowCube = snow_util.annualIMS2SnowCube(c1, c2, verbose = True)
# snowCube

In [41]:
c1 = c1.sel(time = slice(str(c1.time.dt.year[0].values) + '-07-24', str(c1.time.dt.year[0].values) + '-12-31'))
c2 = c2.sel(time = slice(str(c2.time.dt.year[0].values) + '-01-01', str(c2.time.dt.year[0].values) + '-07-23'))

In [42]:
snowCube = xr.concat((c1, c2), dim = 'time').sortby('time') 

In [43]:
crs = snowCube.rio.crs
snowCube = xr.where((snowCube == 1) | (snowCube == 2), 0, snowCube) # Non-snow (1, 2) = 0
snowCube = xr.where((snowCube == 3) | (snowCube == 4), 1, snowCube) # Snow (3, 4) = 1
snowCube.rio.write_crs(crs, inplace = True) # Reapply crs

snowCube

<xarray.DataArray (time: 365, y: 73, x: 47)> Size: 1MB
dask.array<where, shape=(365, 73, 47), dtype=uint8, chunksize=(1, 73, 47), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 376B 4.002e+05 4.012e+05 ... 4.452e+05 4.462e+05
  * y            (y) float64 584B 7.679e+06 7.678e+06 ... 7.608e+06 7.607e+06
  * time         (time) datetime64[ns] 3kB 2022-07-24 2022-07-25 ... 2023-07-23
    variable     <U18 72B 'IMS_Surface_Values'
    spatial_ref  int64 8B 0

# Save as NetCDF

In [44]:
%%time
pputil.downloadNC(snowCube, r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\Alaska\2 IMS\ims_nc', 'IMS_snowCube_winterYear' + str(year1)[2:] + str(year2)[2:] + '.nc', type = '')

CPU times: total: 2.34 s
Wall time: 18.1 s
